In [1]:
## watch -n 1 nvidia-smi
# 터미널에 실행해서 gpu 보면서 실행시키기

#### 1. Import 정리

In [1]:
from detectron2.data import DatasetCatalog, MetadataCatalog

# 1) 리스트 형태로 반환된 Catalog 이름들에서 remove를 이용해 제거
dataset_list = DatasetCatalog.list()
metadata_list = MetadataCatalog.list()

if "totaltext_train" in dataset_list:
    dataset_list.remove("totaltext_train")
    print("'totaltext_train' removed from DatasetCatalog list")


/opt/conda/lib/python3.11/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/opt/conda/lib/python3.11/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


In [2]:
import sys, os, distutils.core
import random
import torch
import math
import copy
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import logging

from detectron2 import model_zoo
from detectron2.config import LazyConfig, instantiate, get_cfg
from detectron2.data import (
    MetadataCatalog,
    DatasetCatalog,
    build_detection_train_loader,
    build_detection_test_loader
)
from detectron2.engine import SimpleTrainer, DefaultPredictor, hooks, DefaultTrainer, HookBase
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, DatasetEvaluators
from detectron2.utils.logger import setup_logger
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.utils.events import CommonMetricPrinter, JSONWriter, TensorboardXWriter, EventStorage
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.structures import BoxMode
import detectron2.data
from detectron2.data import detection_utils as utils
from AdelaiDet.adet.data.augmentation import RandomCropWithInstance
from detectron2.data import build_detection_test_loader


from adet.config import get_cfg as get_adet_cfg  # AdelaiDet 전용 get_cfg

import matplotlib.pyplot as plt

def set_seed(seed=42):
    """Seed 설정 함수"""
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 데이터셋을 train, validation, test로 나누기
def split_dataset(dataset, train_size, val_size, test_size, seed=42):
    random.seed(seed)  # 무작위성을 고정하기 위한 시드 설정
    random.shuffle(dataset)  # 데이터를 무작위로 섞음
    return (
        dataset[:train_size], 
        dataset[train_size:train_size + val_size], 
        dataset[train_size + val_size:]  # 남은 데이터를 테스트로 할당
    )

# Set GPU device
torch.cuda.set_device(0)

/home/jovyan/snukdt/fcos_model/AdelaiDet/adet/modeling/MEInst/MaskEncoding.py:9: FutureWarning: Decorating classes is deprecated and will be disabled in future versions. You should only decorate functions or methods. To preserve the current behavior of class decoration, you can directly decorate the `__init__` method and nothing else.
  @torch.no_grad()


#### 2. 이미지 등록

In [3]:
# Class 1개로만 할 때 적용
def get_custom_dicts(img_dir, json_file):
    with open(json_file) as f:
        data = json.load(f)

    images = {img["id"]: img for img in data["images"]}
    annotations = {}

    # 유지하고 싶은 클래스: Hiddenlens(category_id=2)
    # 매핑: 2 -> 0
    desired_category = 0

    for ann in data["annotations"]:
        image_id = ann["image_id"]
        # category_id가 2가 아닌 경우 모두 제외
        if ann["category_id"] != desired_category:
            continue

        # 남은 category_id=2를 0으로 remap
        ann_copy = ann.copy()
        annotations.setdefault(image_id, []).append(ann_copy)

    dataset_dicts = []
    for image_id, img_info in images.items():
        filtered_annotations = [
            {
                "bbox": ann["bbox"],
                "bbox_mode": BoxMode.XYWH_ABS,
                "category_id": ann["category_id"]
            }
            for ann in annotations.get(image_id, [])
        ]

        if filtered_annotations:
            record = {
                "file_name": os.path.join(img_dir, img_info["file_name"]),
                "image_id": image_id,
                "height": img_info["height"],
                "width": img_info["width"],
                "annotations": filtered_annotations,
            }
            dataset_dicts.append(record)

    # 이 구조 Detectron2에 학습에 사용할 dataset 정보
    return dataset_dicts

#### 3. Crop 적용 Custom Mapper (5~35 pixel)
- 320 x 320으로 Resize되었을 때 불법 카메라가 5~35 pixel 범위로 들어올 수 있도록 설정
    
- 원본 이미지에서 45 pixel 이하인 경우에는 (480, 640)으로 Crop 적용
- 원본 이미지에서 45 pixel 이상인 경우에는 5 ~ 35 pixel 범위로 들어올 수 있도록 Crop할 Width 범위를 구해서 중간값 적용(이 부분은 중간값 적용하지 않고, 다른 방법들 적용이 가능합니다. 저희는 중간 값을 적용했습니다.)
    - 정확한 수식은 cropW_min에서 35를 나눠주고, cropW_max에선느 5를 나눠줘야 되지만, 대략적으로 정규분포 형태를 띄게 하려고 각각 30, 10으로 나눠줬습니다.

- CROP 적용 이후 Dataset에서 0 pixel로 기록된 Hiddenlens가 있는데, 이는 한 개의 이미지에 2개 이상의 Hiddenlens가 있어 CROP 과정에서 잘려서 0 Pixel로 기록된 것입니다.

In [5]:
import copy
import numpy as np
import torch

from detectron2.data import detection_utils as utils
from detectron2.structures import BoxMode

def custom_mapper(dataset_dict):
    """
    이미지를 크롭하고, bbox를 XYWH 형태로 변환한 뒤
    dataset_dict['annotations']와 dataset_dict["image"] 등을 업데이트.

    - 폭 45 이상인 Hiddenlens가 하나라도 있으면 특수 크롭 (3:4 비율)
    - 아니면 (640,480) 크기로 RandomCropWithInstance
    """
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict["file_name"], format="BGR")

    annos = dataset_dict["annotations"]
    boxes_xyxy = [] # 변환된 모든 바운딩 박스를 저장하는 리스트

    # Hiddenlens 폭 리스트 (category_id=0 인 bbox)
    hiddenlens_bbox_widths = []

    # -- A. Hiddenlens(카테고리=0) bbox 폭 계산 --
    for anno in annos:
        # (1) category=0 이면 폭 계산
        if anno["category_id"] == 0:
            # bbox를 XYWH -> XYXY로 변환
            box_xyxy = BoxMode.convert(anno["bbox"], BoxMode.XYWH_ABS, BoxMode.XYXY_ABS)
            w_lens = box_xyxy[2] - box_xyxy[0]
            hiddenlens_bbox_widths.append(w_lens)

        # (2) 모든 bbox XYXY 변환 (transform 에 쓰기 위함)
        box_xyxy_all = BoxMode.convert(anno["bbox"], BoxMode.XYWH_ABS, BoxMode.XYXY_ABS)
        boxes_xyxy.append(box_xyxy_all)

    # -- B. "폭 45 이상"인 Hiddenlens가 하나라도 있으면 특수(수동) 크롭 --
    #     아니면 (모두 45 미만이거나 없음) -> 기존 로직(RandomCropWithInstance) --
    apply_special_crop = any(w >= 45 for w in hiddenlens_bbox_widths)

    # 이미지 높이, 너비
    orig_h, orig_w = image.shape[:2]

    if apply_special_crop:
        # ------------------------------------------------
        # (1) 특수 크롭 (3:4 비율 유지)
        # ------------------------------------------------
        w_lens_max = max(hiddenlens_bbox_widths)

        # cropW 범위 계산
        # Hiddenlens의 폭(w_lens_max)에 따라 크롭 폭을 결정
        cropW_min = (320.0 * w_lens_max) / 30.0
        cropW_max = (320.0 * w_lens_max) / 10.0

        # clip
        cropW_min = max(1, min(cropW_min, orig_w))
        cropW_max = max(1, min(cropW_max, orig_w))

        if cropW_min > cropW_max:
            # 범위 말이 안되면 fallback
            cropW = min(orig_w, 480)
        else:
            # 중간값 적용(여기서 random.uniform도 적용 가능)
            cropW = (cropW_min + cropW_max) * 0.5

        # 3:4 비율 -> 실제 휴대폰에서 이미지를 받는 비율인 3:4 맞춰주기
        cropH = (4.0 / 3.0) * cropW

        # 만약 cropH이 원본보다 크면 clip
        if cropH > orig_h:
            ratio = orig_h / cropH
            cropH = orig_h
            cropW = cropW * ratio

        cropW = int(round(cropW))
        cropH = int(round(cropH))

        # RandomCropWithInstance 적용(객체 중심의 Crop 기법)
        # AdelaiDet에서 제공
        from adet.data.augmentation import RandomCropWithInstance
        special_crop = RandomCropWithInstance(
            crop_type='absolute',
            crop_size=(cropH, cropW),
            crop_instance=True
        )
        transform = special_crop.get_transform(image, boxes_xyxy)

        image_cropped = transform.apply_image(image)
        image_cropped = image_cropped.copy()

        annos_new = []
        for a in annos:
            # 기본 BBox를 XYXY로 변환
            box_xyxy = BoxMode.convert(a["bbox"], BoxMode.XYWH_ABS, BoxMode.XYXY_ABS)
            # BBox가 크롭된 부분을 기준으로 새로운 좌표를 계산
            box_xyxy_2d = np.array([box_xyxy])
            box_xyxy_2d = transform.apply_box(box_xyxy_2d) # 변환 적용
            box_xyxy_out = box_xyxy_2d[0] # 변환된 BBox

            # 유효 bbox 검사
            if (box_xyxy_out[2] > box_xyxy_out[0]) and (box_xyxy_out[3] > box_xyxy_out[1]):
                # 다시 XYWH로 변환
                box_xywh_2d = BoxMode.convert(
                    box_xyxy_2d, BoxMode.XYXY_ABS, BoxMode.XYWH_ABS
                )
                a_new = copy.deepcopy(a)
                a_new["bbox"] = box_xywh_2d[0].tolist()
                a_new["bbox_mode"] = BoxMode.XYWH_ABS  # ← XYWH 모드로 명시
                annos_new.append(a_new)

        new_h, new_w = image_cropped.shape[:2]
        dataset_dict["annotations"] = annos_new
        dataset_dict["height"] = new_h
        dataset_dict["width"]  = new_w

        # (C,H,W) 순서로 변환
        image_chw = image_cropped.transpose(2, 0, 1).copy()
        dataset_dict["image"] = torch.as_tensor(image_chw, dtype=torch.float32)

        # 최종 instances
        annos_transformed = [
            utils.transform_instance_annotations(
                obj, transforms=[], image_size=(new_h, new_w)
            )
            for obj in annos_new
        ]
        from detectron2.data import detection_utils
        dataset_dict["instances"] = detection_utils.annotations_to_instances(
            annos_transformed, (new_h, new_w)
        )
        return dataset_dict

    else:
        # ------------------------------------------------
        # (2) 폭 45 미만 → 기존 RandomCropWithInstance
        # ------------------------------------------------
        from adet.data.augmentation import RandomCropWithInstance
        crop_augment = RandomCropWithInstance(
            crop_type='absolute', 
            crop_size=(640, 480),
            crop_instance=True
        )
        transform = crop_augment.get_transform(image, boxes_xyxy)
        image_cropped = transform.apply_image(image)
        image_cropped = image_cropped.copy()

        annos_new = []
        for a in annos:
            box_xyxy = BoxMode.convert(a["bbox"], BoxMode.XYWH_ABS, BoxMode.XYXY_ABS)
            box_xyxy_2d = np.array([box_xyxy])
            box_xyxy_2d = transform.apply_box(box_xyxy_2d)
            box_xyxy_out = box_xyxy_2d[0]

            # 유효 bbox 검사
            if (box_xyxy_out[2] > box_xyxy_out[0]) and (box_xyxy_out[3] > box_xyxy_out[1]):
                # XYWH 재변환
                box_xywh_2d = BoxMode.convert(
                    box_xyxy_2d, BoxMode.XYXY_ABS, BoxMode.XYWH_ABS
                )
                a_new = copy.deepcopy(a)
                a_new["bbox"] = box_xywh_2d[0].tolist()
                a_new["bbox_mode"] = BoxMode.XYWH_ABS  # ← XYWH 모드 명시
                annos_new.append(a_new)

        new_h, new_w = image_cropped.shape[:2]
        dataset_dict["annotations"] = annos_new
        dataset_dict["height"] = new_h
        dataset_dict["width"]  = new_w

        image_chw = image_cropped.transpose(2, 0, 1).copy()
        dataset_dict["image"] = torch.as_tensor(image_chw, dtype=torch.float32)

        annos_transformed = [
            utils.transform_instance_annotations(
                obj, transforms=[], image_size=(new_h, new_w)
            )
            for obj in annos_new
        ]
        from detectron2.data import detection_utils
        dataset_dict["instances"] = detection_utils.annotations_to_instances(
            annos_transformed, (new_h, new_w)
        )
        return dataset_dict

# 크롭 적용
class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)

    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        return COCOEvaluator(dataset_name, cfg, True, output_dir=cfg.OUTPUT_DIR)

#### 4. 흔들리거나 식별이 어려운 이미지 제거

In [9]:
# 삭제할 인덱스 정의
delete_indices = set()

# 첫 번째 그룹: +1 적용
delete_indices.update([178 + 1, 249 + 1, 263 + 1, 63 + 1, 340 + 1, 357 + 1, 358 + 1, 
                       359 + 1, 360 + 1, 367 + 1, 413 + 1, 481 + 1,
                       20 + 1, 35 + 1, 53 + 1, 54 + 1, 58 + 1, 61 + 1, 136 + 1, 147 + 1, 
                       152 + 1, 215 + 1, 247 + 1, 248 + 1, 255 + 1, 262 + 1, 268 + 1, 
                       269 + 1, 299 + 1, 332 + 1, 355 + 1, 390 + 1, 399 + 1, 483 + 1, 492 + 1])

# 두 번째 그룹: +500 적용
delete_indices.update([
    205 + 500, 217 + 500, 247 + 500, 474 + 500, 344 + 500, 
    362 + 500, 481 + 500, 499 + 500,
    17 + 500, 30 + 500, 34 + 500, 55 + 500, 58 + 500, 74 + 500, 
    106 + 500, 148 + 500, 160 + 500, 199 + 500, 211 + 500, 260 + 500, 
    263 + 500, 264 + 500, 281 + 500, 285 + 500, 288 + 500, 321 + 500, 
    359 + 500, 372 + 500, 377 + 500])

# 세 번째 그룹: +1000 적용
delete_indices.update([
    236 + 1000, 245 + 1000, 267 + 1000, 314 + 1000, 334 + 1000, 
    346 + 1000, 353 + 1000, 382 + 1000, 483 + 1000, 499 + 1000,
    14 + 1000, 19 + 1000, 22 + 1000, 25 + 1000, 43 + 1000, 47 + 1000, 
    54 + 1000, 96 + 1000, 106 + 1000, 117 + 1000, 138 + 1000, 140 + 1000, 
    141 + 1000, 149 + 1000, 177 + 1000, 189 + 1000, 197 + 1000, 198 + 1000, 
    204 + 1000, 205 + 1000, 231 + 1000, 232 + 1000, 234 + 1000, 235 + 1000, 
    277 + 1000, 280 + 1000, 294 + 1000, 348 + 1000, 349 + 1000, 358 + 1000, 
    362 + 1000, 419 + 1000, 485 + 1000, 492 + 1000, 494 + 1000, 498 + 1000])

print(len(delete_indices))

110


#### 5. Crop 적용 후 새로운 파일 생성

In [8]:
import os
import cv2
import json
import copy
import numpy as np
import torch

# detectron2 관련 임포트
from detectron2.structures import BoxMode
from detectron2.data import detection_utils as utils

# ---------------------------------------------------
# (1) Dataset 읽어오기 (get_custom_dicts 등)
# ---------------------------------------------------
json_file = "./instances_default.json"
img_dir   = "./images"
all_data  = get_custom_dicts(img_dir, json_file)
all_data = [d for d in all_data if d["image_id"] not in delete_indices]

# ---------------------------------------------------
# (2) 크롭된 이미지를 output 디렉토리에 저장 + 새 어노테이션 JSON 생성
# ---------------------------------------------------
def save_cropped_dataset(dataset_dicts, output_dir):
    """
    dataset_dicts: Detectron2에서 사용하는 dataset_dict 형식 (list of dict)
    output_dir   : 크롭된 이미지를 저장할 폴더 경로
    """
    os.makedirs(output_dir, exist_ok=True)

    # COCO JSON 형식을 구성하기 위한 변수들
    new_images = []
    new_annotations = []
    next_ann_id = 1  # 새로 매길 annotation id
    category_id_map = {}  # category id 매핑 (보통은 1개 클래스라면 그대로 0 또는 1 등으로 사용)

    # categories (예: 클래스가 1개라면 아래처럼)
    categories = [
        {"id": 0, "name": "Hiddenlens"}
    ]

    for idx, dataset_dict in enumerate(dataset_dicts):
        # custom_mapper 를 호출해 크롭 + bbox 재계산
        transformed_dict = custom_mapper(dataset_dict)

        # (2-1) 이미지 추출 (torch -> numpy)
        cropped_img_chw = transformed_dict["image"]  # (C,H,W)
        cropped_img_hwc = cropped_img_chw.permute(1, 2, 0).numpy()  # (H,W,C)

        # BGR -> 저장 시 그대로 써도 되고, 혹은 RGB->BGR 변환 여부 확인
        # 여기서는 custom_mapper에서 utils.read_image(image, format="BGR")를 썼으므로
        # cropped_img_hwc는 이미 BGR일 가능성이 높음. 필요한 경우 변환
        # cropped_img_hwc = cv2.cvtColor(cropped_img_hwc, cv2.COLOR_RGB2BGR)

        # (2-2) 새 파일명 지정 후 저장
        # ex) "cropped_00001.jpg" 와 같이 저장
        new_filename = f"cropped_{idx:05d}.jpg"
        out_path = os.path.join(output_dir, new_filename)
        cv2.imwrite(out_path, cropped_img_hwc)

        # (2-3) COCO JSON용 images 정보 추가
        new_image_id = idx + 1  # 새 image_id (1부터 부여)
        new_w = transformed_dict["width"]
        new_h = transformed_dict["height"]
        new_images.append({
            "id": new_image_id,
            "file_name": new_filename,
            "width": new_w,
            "height": new_h
        })

        # (2-4) COCO JSON용 annotations 정보 추가
        #       custom_mapper를 거친 후의 bbox가 dataset_dict["annotations"]에 있음
        annos = transformed_dict["annotations"]
        for anno in annos:
            # anno["bbox"]는 XYWH_ABS 형태
            # category_id도 mapper에서 이미 0으로 remap 되어있음 (Hiddenlens)
            new_annotations.append({
                "id": next_ann_id,
                "image_id": new_image_id,
                "category_id": anno["category_id"],
                "bbox": anno["bbox"].tolist(),  # [x, y, w, h]
                "area": anno["bbox"][2] * anno["bbox"][3],  # w*h
                "iscrowd": 0
            })
            next_ann_id += 1

    # (2-5) 최종 COCO json 만들기
    # 필요한 필드만 간단히 생성. (라이센스 정보, segmentations 등은 생략 예시)
    coco_output = {
        "images": new_images,
        "annotations": new_annotations,
        "categories": categories,
    }

    # JSON 저장
    with open(os.path.join(output_dir, "cropped_annotations.json"), "w", encoding="utf-8") as f:
        json.dump(coco_output, f, ensure_ascii=False, indent=2)

    print(f"Done! Cropped dataset & annotations saved in: {output_dir}")

# ---------------------------------------------------
# 실제 실행
# ---------------------------------------------------
output_dir = "./cropped_output"
save_cropped_dataset(all_data, output_dir)


Done! Cropped dataset & annotations saved in: ./cropped_output


#### 6. 새롭게 만들어진 파일을 이용해 변환 잘 되었는지 시각화

In [ ]:
import os
import json
import cv2
import matplotlib.pyplot as plt

# --------------- 사용자 환경에 맞게 설정 ---------------
annotation_file = "./cropped_output/cropped_annotations_xywh.json"
images_dir      = "./cropped_output"
hiddenlens_id   = 0   # "Hiddenlens" 클래스 id
# -----------------------------------------------------

# (1) JSON 로드
with open(annotation_file, "r", encoding="utf-8") as f:
    dataset = json.load(f)

# images, annotations를 딕셔너리로 매핑
images_info = {img["id"]: img for img in dataset["images"]}

annotations_by_image = {}
for ann in dataset["annotations"]:
    image_id = ann["image_id"]
    annotations_by_image.setdefault(image_id, []).append(ann)

# (2) 이미지별로 로드 & 시각화
for image_id, image_meta in images_info.items():
    file_name = image_meta["file_name"]
    file_path = os.path.join(images_dir, file_name)

    # OpenCV로 이미지 읽기 (BGR)
    image_bgr = cv2.imread(file_path)
    if image_bgr is None:
        print(f"[WARN] Could not load image: {file_path}")
        continue

    # OpenCV에서 shape (H, W, C) 획득
    h_img, w_img = image_bgr.shape[:2]

    # 화면(터미널)에 이미지/크기 정보 출력
    print(f"---- Image ID: {image_id} ----")
    print(f" file_name: {file_name}")
    print(f" Image Width={w_img}, Height={h_img}")

    # Bounding Box 시각화를 위해, 이미지를 복사해서 사용
    vis_image_bgr = image_bgr.copy()

    # 해당 이미지의 어노테이션들
    if image_id not in annotations_by_image:
        # 어노테이션이 없는 경우 바로 넘어감
        plt.figure(figsize=(8, 6))
        plt.imshow(cv2.cvtColor(vis_image_bgr, cv2.COLOR_BGR2RGB))
        plt.title(f"Image ID {image_id}: No Annotations")
        plt.axis("off")
        plt.show()
        continue

    for ann in annotations_by_image[image_id]:
        # bbox = [x, y, w, h], XYWH 형식
        x, y, w_bb, h_bb = ann["bbox"]
        x2 = x + w_bb 
        y2 = y + h_bb
        
        # 초록색 사각형으로 표시
        cv2.rectangle(
            vis_image_bgr,
            (int(x), int(y)),
            (int(x2), int(y2)),
            color=(0, 255, 0),
            thickness=2
        )

        # Hiddenlens 클래스면 폭, 높이만 출력
        if ann["category_id"] == hiddenlens_id:
            print(f"  [Hiddenlens] BBox => width={w_bb:.1f}, height={h_bb:.1f}")

    # (3) Matplotlib으로 시각화 (BGR→RGB 변환)
    vis_image_rgb = cv2.cvtColor(vis_image_bgr, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    plt.imshow(vis_image_rgb)
    plt.title(f"Image ID {image_id}: {file_name}")
    plt.axis("off")
    plt.show()


#### 7. 기존 Dataset의 Json 파일과 같은 형식으로 가져오기
- 완전히 동일한 형식으로 가져오지 않고 학습에 필요한 부분만 Json 파일로 만들었습니다.

In [13]:
import json
import os

def convert_bbox_xyxy_to_xywh(json_path, output_path=None):
    """
    cropped_annotations.json 안의 모든 bbox가 [x1, y1, x2, y2] 인 것으로 가정.
    이를 [x, y, w, h] 으로 바꾸고, area를 다시 계산해서 저장.

    Args:
        json_path   : 원본 JSON 경로 (예: "./cropped_annotations.json")
        output_path : 변환 완료 후 저장할 JSON 경로
                     None 이면, 원본 파일에 덮어씀
    """
    if output_path is None:
        # 덮어쓰지 않으려면 다른 이름으로 저장하세요.
        output_path = json_path

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    annotations = data.get("annotations", [])

    for ann in annotations:
        bbox = ann.get("bbox", None)
        if bbox is None or len(bbox) < 4:
            continue

        # bbox = [x1, y1, x2, y2] 이라고 가정
        x1 = bbox[0]
        y1 = bbox[1]
        x2 = bbox[2]
        y2 = bbox[3]

        # 새로 w, h 계산
        w = x2 - x1
        h = y2 - y1

        # bbox 업데이트: [x, y, w, h] 로 변환
        ann["bbox"] = [x1, y1, w, h]

        # area 다시 계산
        # 만약 음수(잘못된 bbox)일 때는 0 처리
        if w < 0 or h < 0:
            new_area = 0
        else:
            new_area = w * h

        ann["area"] = new_area

    # 업데이트 완료 후 저장
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"Done! Saved to: {output_path}")


# 실제 사용 예시
if __name__ == "__main__":
    # 원본 JSON 파일 경로
    json_input = "./cropped_output/cropped_annotations.json"
    # 결과 저장 경로 (원본에 덮어쓰려면 json_input과 동일하게 설정 가능)
    json_output = "./cropped_output/cropped_annotations_xywh.json"

    convert_bbox_xyxy_to_xywh(json_input, json_output)


Done! Saved to: ./cropped_output/cropped_annotations_xywh.json


In [14]:
import json

def update_categories(json_path, output_path=None):
    """
    cropped_annotations.json 파일을 로드하여,
    categories[0]에 {id=1, name='Hiddenlens', supercategory=''} 를 설정한 뒤 저장한다.
    
    :param json_path: (str) 수정할 JSON 파일 경로
    :param output_path: (str) 수정 결과를 저장할 경로 (None이면 원본에 덮어쓰기)
    """
    if output_path is None:
        output_path = json_path  # 덮어쓰기

    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # categories 첫 번째 항목 수정
    if "categories" in data and len(data["categories"]) > 0:
        data["categories"][0]["id"] = 1
        data["categories"][0]["name"] = "Hiddenlens"
        data["categories"][0]["supercategory"] = ""

    # 수정된 JSON 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"Updated categories[0] in: {output_path}")

# 사용 예시
if __name__ == "__main__":
    json_file = "./cropped_output/cropped_annotations_xywh.json"
    update_categories(json_file)  # 원본에 덮어쓰기


Updated categories[0] in: ./cropped_output/cropped_annotations_xywh.json


In [4]:
!apt-get update
!apt-get install zip -y
!yum install zip -y

Reading package lists... Done
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/bin/bash: line 1: yum: command not found


In [5]:
import shutil

# 폴더 "cropped_output"을 "cropped_output.zip"으로 압축
shutil.make_archive("cropped_output", "zip", "cropped_output")

print("Done! Created cropped_output.zip")


Done! Created cropped_output.zip
